In [89]:
from itertools import cycle

import numpy as np
import numpy.random as rn


DEBUG = 2
def debug(level, *args):
    if DEBUG > level:
        print(*args)

In [90]:
def win_tictactoe(board):
    '''Return 0 for no win, otherwise i if player i wins for board vector row major ordering.'''
    debug(10, 'board [{}]'.format(board))
    for player in [1, 2]:
        #Player win by rows.
        player_win = np.any(np.all(
            board.reshape(3, 3) == player,
            axis=1))
        debug(10, 'board 3 by 3 [{}]'.format(board.reshape(3, 3)))
        debug(10, 'player [{}] win by row [{}]'.format(player, player_win))
        
        #Player win by columns.
        player_win = player_win or np.any(np.all(
            board.reshape(3, 3) == player,
            axis=0))
        debug(10, 'board 3 by 3 [{}]'.format(board.reshape(3, 3)))
        debug(10, 'player [{}] win by columns [{}]'.format(player, player_win))
        
        #Player win by diagonals.
        player_win = player_win or np.any(np.all(np.vstack([
            board.reshape(3, 3)[[0,1,2],[0,1,2]] == player,
            board.reshape(3, 3)[[0,1,2],[2,1,0]] == player]),
            axis=0))
        
        debug(10, 'board 3 by 3 [{}]'.format(board.reshape(3, 3)))
        debug(10, 'player [{}] win by diagonal [{}]'.format(player, player_win))
        
        debug(10, 'diag board slices [{}]'.format(np.vstack([
            board.reshape(3, 3)[[0,1,2],[0,1,2]] == player,
            board.reshape(3, 3)[[0,1,2],[2,1,0]] == player])))
        debug(10, 'diag board player [{}] slices [{}]'.format(player, np.vstack([
            board.reshape(3, 3)[[0,1,2],[0,1,2]] == player,
            board.reshape(3, 3)[[0,1,2],[2,1,0]] == player])))
        
        
        if player_win:
            return player
        
    return 0

def tie_tictactoe(board):
    '''Return true if board is a tie.'''
    
    return np.all(board != 0)

def valid_moves(board):
    return np.where(board == 0)[0]


class RandomPlayer:
    def __init__(self, player):
        self.player = player
        
    def action(self, board):
        action = rn.choice(valid_moves(board))
        return (self.player, action)
    
class Tictactoe:
    def __init__(self, player1, player2):
        self.board = np.zeros(9)
        self.players = [player1, player2]
        self.action_log = list()
        self.result = 'playing'
        
    def play(self):
        for player in cycle(self.players):
            action = player.action(self.board)
            self.update(*action)
            self.action_log.append(action)
            
            if win_tictactoe(self.board) != 0:
                self.result = 'winner_player_{}'.format(player.player)
            elif tie_tictactoe(self.board):
                self.result = 'tie'
            
            debug(1, self)
            
            if self.result != 'playing':
                break
        
    def update(self, player, location):
        self.board[location] = player
        
        
    def __repr__(self):
        return '\n'.join([
                '->'.join(map(str, self.action_log)),
                str(self.board.reshape(3,3)),
                self.result
            ])

In [91]:
player_1_winning_boards = np.array([
        [1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 1, 0, 1, 0, 1, 0, 0]
    ])
player_2_winning_boards = np.copy(player_1_winning_boards)
player_2_winning_boards[player_2_winning_boards == 1] = 2

In [92]:
for b in player_1_winning_boards:
    print(win_tictactoe(b))
for b in player_2_winning_boards:
    print(win_tictactoe(b))
print(win_tictactoe(np.zeros(9)))
print(tie_tictactoe(np.zeros(9)))
print(tie_tictactoe(np.ones(9)))
for b in player_1_winning_boards:
    print(valid_moves(b))
    
print(not tie_tictactoe(np.zeros(9)) == (len(valid_moves(np.zeros(9))) > 0))
print(tie_tictactoe(np.ones(9)) == (len(valid_moves(np.ones(9))) == 0))

player1 = RandomPlayer(1)
player2 = RandomPlayer(2)
for b in player_1_winning_boards:
    print(player1.action(b))
    print(player2.action(b))

1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
0
False
True
[3 4 5 6 7 8]
[0 1 2 6 7 8]
[0 1 2 3 4 5]
[1 2 4 5 7 8]
[0 2 3 5 6 8]
[0 1 3 4 6 7]
[1 2 3 5 6 7]
[0 1 3 5 7 8]
True
True
(1, 5)
(2, 6)
(1, 2)
(2, 6)
(1, 0)
(2, 0)
(1, 1)
(2, 5)
(1, 2)
(2, 3)
(1, 0)
(2, 4)
(1, 5)
(2, 6)
(1, 0)
(2, 0)


In [93]:
game = Tictactoe(RandomPlayer(1), RandomPlayer(2))
game.play()
game

(1, 6)
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]]
playing
(1, 6)->(2, 7)
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  2.  0.]]
playing
(1, 6)->(2, 7)->(1, 2)
[[ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  0.]]
playing
(1, 6)->(2, 7)->(1, 2)->(2, 8)
[[ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]
playing
(1, 6)->(2, 7)->(1, 2)->(2, 8)->(1, 0)
[[ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]
winner_player_1


(1, 6)->(2, 7)->(1, 2)->(2, 8)->(1, 0)
[[ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]
winner_player_1

In [94]:
DEBUG=11
win_tictactoe(game.board)

board [[ 1.  0.  1.  0.  0.  0.  1.  2.  2.]]
board 3 by 3 [[[ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]]
player [1] win by row [False]
board 3 by 3 [[[ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]]
player [1] win by columns [False]
board 3 by 3 [[[ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]]
player [1] win by diagonal [True]
diag board slices [[[ True False False]
 [ True False  True]]]
diag board player [1] slices [[[ True False False]
 [ True False  True]]]


1

In [35]:
game.__dict__

{'action_log': [(1, 1),
  (2, 0),
  (1, 3),
  (2, 5),
  (1, 2),
  (2, 6),
  (1, 8),
  (2, 7),
  (1, 4)],
 'board': array([ 2.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,  1.]),
 'players': [<__main__.RandomPlayer at 0x1fd4231cf98>,
 'result': 'tie'}